In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import traceback
import re
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
input_file_path = 'D:\Program Files\Documents\CMU\BAR_EMA\output_data\data_draft.csv'
output_file_path = 'D:\Program Files\Documents\CMU\BAR_EMA\output_data\data_draft_v2.csv'

In [3]:
df = pd.read_csv(input_file_path)
df.head(3)

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,MR_wake,MR_rise,MR_bed,MR_FlSlp,MR_SlpQlt,MR_DrnkNum,MR_DrnkDur,MR_rlx,MR_hngvr,MR_Embrssd,MR_agrsv,MR_vom,MR_hrt,MR_rmbr,MR_work,MR_rude,MR_soc,MR_exprs,MR_guilt,MR_expctd,MR_mood,MR_mrng,MR_prmsd,MR_more,MR_fght,MR_enrgtc,MR_PsdOut,MR_MsdClss,MR_Impls,MR_sleep,MR_weed,MR_plan,chr,anoy,upbt,sad,irit,hap,bor,cnt,RP_enrgz,RP_excit,RP_sedat,RP_slotht,RP_slug,RP_up,RP_crntloc,RP_othrloc,RP_wthothr,RP_NumOth,RP_OthTyp,RP_OthWho,RP_EnjInt,RP_ActCld,RP_ActAcc,RP_OthCld,RP_OthAcc,RP_Vrtl,RP_AlnLke,RP_AlnGd,RP_WshOth,RP_lnly,RP_rjct,RP_incl,RP_acpt,RP_vrtint,RP_vrtoth,RP_vrtwho,RP_vrtenj,RP_vrtcld,RP_VrtAcc,RP_VrtOthCld,RP_VrtOthAcc,RP_NotInt,RP_WshInt,RP_lonely,RP_reject,RP_included,RP_accepted,RP_said,RP_spent,RP_impt,RP_spur,RP_Drnk
0,2001,1,1,Thu,5ab425aa2c9b2f5baaa69eaf,Submission,5a8381835893ca7070656612,Morning Reports,Assessment,2018-03-22,17:49:33,2018-03-22,17:52:41,2018-03-22,05:00:00,07:30:00,07:40:00,12:00:00,1.0,4.0,1.0,CONDITION_SKIPPED,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,71.0,27.0,79.0,21.0,9.0,69.0,11.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,1,1,Fri,5ab528815831424890f49c8c,Submission,5a8381835893ca7070656612,Daytime Surveys,Assessment,2018-03-23,12:15:03,2018-03-23,12:17:04,2018-03-23,12:14:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,14.0,76.0,15.0,15.0,82.0,33.0,78.0,78.0,55.0,14.0,14.0,27.0,84.0,6.0,CONDITION_SKIPPED,1.0,11,"4,5,6,7",CONDITION_SKIPPED,59,41,42,58,35,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,33.0,67.0,2.0
2,2001,1,1,Fri,5ab53c5fe15db24566df3a98,Submission,5a8381835893ca7070656612,Daytime Surveys,Assessment,2018-03-23,13:40:33,2018-03-23,13:41:51,2018-03-23,13:39:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,13.0,77.0,13.0,29.0,83.0,17.0,76.0,82.0,35.0,27.0,13.0,24.0,75.0,6.0,CONDITION_SKIPPED,1.0,2,5,CONDITION_SKIPPED,71,82,19,69,82,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,0.0,0.0,2.0


### TO DO
- Fill in Daytime Survey values
- MR_DrnkDur Column
- MR_wake

In [4]:
# Create a progress bar using tqdm
progress_bar = tqdm(total=len(df[df['SurvName'] == 'Daytime Surveys']))

# Iterate over rows with SurvName equal to 'Daytime Surveys'
for index, row in df[df['SurvName'] == 'Daytime Surveys'].iterrows():
    date = row['SubDate']
    
    morning_df = df[(df['SubDate'] == date) & (df['SurvName'] == 'Morning Reports')]
    if len(morning_df)==0:
        continue
    
    morning_reports_row = df[(df['SubDate'] == date) & (df['SurvName'] == 'Morning Reports')].iloc[0]

    # Fill NaN values in the 'Daytime Surveys' row using corresponding 'Morning Reports' row
    for column in df.columns:
        if pd.isna(row[column]):
            df.loc[index, column] = morning_reports_row[column]

    progress_bar.update(1)

progress_bar.close()

100%|█████████▉| 22458/22497 [05:22<00:00, 69.60it/s]


In [5]:
def normalize_duration(duration_str):
    result = ''
    if not duration_str or duration_str=='CONDITION_SKIPPED' or duration_str=='nan' or pd.isna(duration_str):
        result = 'CONDITION_SKIPPED'
    
    if result != 'CONDITION_SKIPPED':
        duration_str = str(duration_str)
        # Use regular expressions to extract hours and minutes
        hours = 0
        minutes = 0

        # Extract hours
        hours_match = re.search(r'(\d+)\s*hours?|(\d+)\s*hrs?|(^\d+)\s*\d*$', duration_str)
        if hours_match:
            for group in hours_match.groups():
                if group is not None:
                    hours = int(group)
                    break

        # Extract minutes
        minutes_match = re.search(r'(\d+)\s*minutes+|(\d+)\s*min+|\s+(\d+$)', duration_str)
        if minutes_match:
            for group in minutes_match.groups():
                if group is not None:
                    minutes = int(group)
                    break
        result = f'{hours} hours {minutes} minutes'

    # # Calculate the total time in minutes
    # total_minutes = hours * 60 + minutes
    
#     print(f'{duration_str} ----> {result}')

    return f'{result}'

# Apply the normalization function to the DataFrame
df['MR_DrnkDur'] = df['MR_DrnkDur'].apply(normalize_duration)

In [6]:
df.to_csv(output_file_path,index=False)